# Testing `pylibs`


In [2]:
import testutils as u
import numpy as np

In [ ]:
p = u.testPlama_CuSn() # plasma with default settings

import matplotlib.pyplot as plt
plt.style.use('ggplot')

plt.figure()
y = p.computeSpectrum(350., 650., 1000)
plt.plot(y.c('lambda'), y.c('sum'))
plt.show()



def sepctrum_test():
    ref = {
        510.554 : 'cu-1',
        515.324 : 'cu-1',
        521.820 : 'cu-1',
        380.1011: 'sn-1',
        452.4734: 'sn-1', 
        533.2339: 'sn-2', 
        556.1910: 'sn-2', 
        558.8815: 'sn-2', 
        579.8860: 'sn-2', 
        645.3542: 'sn-2', 
        684.4186: 'sn-2',
    }

    s = _test_cu_sn_spectrum()
    q = QAnalyser(s, list(s.attrs['plasma'].comp.keys()))
    q.setReferenceLines(ref)

    q.searchPeaks(match=1, dist_ub=1.)

    # q._lines.print()

    q.makeBoltzPlane(1., 1.e+17, fmt = 'saha')

    b = q.getBoltzPlane()
    # t = b.getPoints()
    # t.print()

    # b.getLineParam().print()
    b.estimateTe('cu')

    # import matplotlib.pyplot as plt
    # plt.style.use('ggplot')

    # plt.figure()
    # sp = ... 
    # for elem in q._elem_idx.keys():
    #     t = b.getPoints(elem, sp)
    #     plt.plot(t.c('x'), t.c('y'), '+')
    # plt.show()
    

if __name__ == "__main__":
    sepctrum_test()